Идея - написать два класса:
первый считывает из файла конкретный автомат и считает длину периода (в первой версии) до первой единицы
второй будет генерировать файлы с заданными параметрами (под каждый вид автомата пишется отдельный генератор)

Формат файла:
n \t m \t k
начальное состояние \t начальная строка

опциональные строки с алфавитами: по умолчанию $Q = \{0,...,n-1 \}, \Gamma = \{1,...,k\}, 0 = \lambda$, $B = \{0, 1\}$

функция переходов: Q \t Gamma \t Q, Q \t \lambda \t Q

функция выхода: Q \t Gamma \t B, Q \t \lambda \t B

функция записи в магазин: ...


## Основной класс

In [20]:
class Pushdown(object):
    def __init__(self, fileName):
        sep = '\n'
        with open(fileName) as fin:
            text = fin.read()
            
        lines = text.split(sep)
        sep = '\t'
        nmk = lines[0].split(sep)
        self.n = int(nmk[0]) 
        self.m = int(nmk[1]) 
        self.k = int(nmk[2])
        #print self.n, self.m, self.k
        
        second = lines[1].split(sep)
        
        self.state = int(second[0])
        self.stack = second[1]
        #print self.state, len(self.stack), self.stack
        #print lines[2: 2 + self.n* (self.m + 1) + 1]
        self.phi = self.createMap(lines[2: 2 + self.n* (self.m + 1) + 1])
        #print self.phi
        #print lines[3 + self.n* (self.m + 1): 4 + 2 * self.n* (self.m + 1)]
        self.psi = self.createMap(lines[3 + self.n* (self.m + 1): 4 + 2 * self.n* (self.m + 1)])
        #print self.psi
        #print lines[4 + 2*self.n* (self.m + 1): 5 + 3 * self.n* (self.m + 1)]
        self.eta = self.createMap(lines[4 + 2*self.n* (self.m + 1): 5 + 3 * self.n* (self.m + 1)])
        #print self.eta
        
    def createMap(self, lines):
        #print lines[0]
        result = {}
        for i in range(self.n):
            result[i] = {}
        for line in lines[1:]:
            split_line = line.split('\t')
            #print split_line
            result[int(split_line[0])][int(split_line[1])] = split_line[2] 
        return result
    
    def calculatePeriod(self, verbose=0):
        result = 1
        while(self.step() != 1):
            result = 1
        if verbose > 0:
            print(result, self.state, self.stack)
        while(self.step() != 1):
            result += 1
            if verbose > 0:
                 print(result, self.state, self.stack)
            
        return result
    
    def step(self):
        #print self.state, self.z(), self.psi
        result = self.psi[self.state][self.z()]
        next_state = int(self.phi[self.state][self.z()])
        next_stack = self.nextStack()
        self.state = next_state
        self.stack = next_stack
        return int(result)
    
    def z(self):
        if len(self.stack) == 0:
            return 0
        else:
            return int(self.stack[-1])
        
    def nextStack(self):
        if self.eta[self.state][self.z()] == "0":
            return self.stack[:-1]
        else:
            return self.stack[:-1] + self.eta[self.state][self.z()]

In [27]:
path = "C:/Users/iivanov/Desktop/soft/automata/"
pushdown = Pushdown(path + "max_pda_n_3_m_2_k_3.txt")
print(pushdown.calculatePeriod(0))


31


In [25]:
print (theoryEstimateSimpleCase(3, 3))
print (theoryEstimate(3, 2, 3))

31.0

In [47]:
class MaxPushdownGenerator(object):
    def __init__(self, n, m, k, fileName):
        self.n = n
        self.m = m
        self.k = k
        with open(fileName, 'w') as fout:
            fout.write(str(n))
            fout.write('\t')
            fout.write(str(m))
            fout.write('\t')
            fout.write(str(k))
            fout.write('\n')
            fout.write('0')
            fout.write('\t\n')
            
            fout.write('phi\n')
            for q in range(n):
                for z in range(m+1):
                    fout.write(str(q) + '\t' + str(z) + '\t' + str(self.phi(q,z)) + '\n')
 
            fout.write('psi\n')
            for q in range(n):
                for z in range(m+1):
                    fout.write(str(q) + '\t' + str(z) + '\t' + str(self.psi(q,z)) + '\n')
            
            fout.write('eta\n')
            for q in range(n):
                for z in range(m+1):
                    fout.write(str(q) + '\t' + str(z) + '\t' + str(self.eta(q,z)) + '\n')
    
    def phi(self, q, z):
        if z == self.m and q != 0:
            return q-1
        if z == self.m - 1 and q != self.n - 1:
            return q + 1
        return q
    
    def psi(self, q, z):
        if q == 0 and z == 0:
            return 1
        return 0
    
    def eta(self, q, z):
        if z == 0:
            return repeat(self.k, '1')
        if z == self.m:
            return '0';
        if z == self.m - 1 and q == self.n - 1:
            return '0'
        if z == self.m - 1 and q != self.n - 1:
            return str(self.m) + repeat(self.k - 1, '1')
        return repeat(k, str(z + 1))
        
def repeat(ntimes, symbol):
    result  = ''
    for i in range(ntimes):
        result += symbol
    return result

In [107]:
def theoryEstimateSimpleCase(n, k):
    if k == 2:
        return 4*n - 1
    else:
        return 1 + k*((k-1)**n + (k-1)**(n-1) - 2)/ (1.0 * k - 2)
    
def theoryEstimate(n, m, k):
    if k == 2 and m == 2:
        return 4*n - 1
    else:
        m = m - 1
        return (k**m - 1)/(k-1) + k**m * ( (1+k**(m-1))*((k**m - k**(m-1))**(n-1) - 1)/(k**m - k**(m-1) - 1) + (k**m - k**(m-1))**(n-1) )
    

In [110]:

path = "C:/Users/iivanov/Desktop/soft/automata/"
n = 3
m = 2
k = 2

fileName = "new_" + str(n) + "_" + str(m) + "_" + str(k) + ".txt"
newFileGenerator = MaxPushdownGenerator(n, m ,k, path + fileName)
pushdown = Pushdown(path + fileName)
print(pushdown.calculatePeriod(0))
#print (theoryEstimateSimpleCase(n, k))
print (theoryEstimate(n, m, k))

11
11


In [111]:
path = "C:/Users/iivanov/Desktop/soft/automata/"

for n in range(2, 5):
    for k in range(2, 5):
        for m in range(2, 5):
            fileName = "new_" + str(n) + "_" + str(m) + "_" + str(k) + ".txt"
            newFileGenerator = MaxPushdownGenerator(n, m ,k, path + fileName)
            pushdown = Pushdown(path + fileName)
            if pushdown.calculatePeriod(0) != theoryEstimate(n, m, k):
                print(n, m , k)
